# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
#pip install gmaps

In [12]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

from ipywidgets.embed import embed_minimal_html

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_df = pd.read_csv("output/cities.csv")
city_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp in F,Min Temp in F,Wind Speed,Temp in F,Feels Like
0,935215,Saint-Philippe,38,RE,1604615440,64,-21.36,55.77,71.60,68.00,4.70,68.92,68.22
1,2126710,Beringovskiy,100,RU,1604615440,90,63.05,179.32,38.21,38.21,13.31,38.21,27.70
2,2630299,Hofn,19,IS,1604615440,70,64.25,-15.21,44.85,44.85,40.89,44.85,18.82
3,1283334,Birin,2,NP,1604615440,95,26.89,88.07,49.21,49.21,3.65,49.21,46.67
4,2163355,Hobart,75,AU,1604615377,50,-42.88,147.33,57.00,55.00,17.22,55.85,43.47
...,...,...,...,...,...,...,...,...,...,...,...,...,...
573,777019,Vardø,75,NO,1604615560,85,70.37,31.11,33.80,33.80,12.75,33.80,22.73
574,4771808,Martinsville,1,US,1604615446,54,36.69,-79.87,60.01,57.20,2.15,58.71,55.71
575,1633034,Palu,100,ID,1604615561,76,-0.89,119.87,78.57,78.57,2.39,78.57,85.03
576,7535681,Saint-Agapit,75,CA,1604615561,70,46.56,-71.44,51.80,46.40,2.24,49.98,46.62


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = city_df[["Latitude", "Longitude"]]
humidity = city_df["Humidity"]

figure = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(locations, 
                                    weights = humidity, 
                                    dissipating=False, 
                                    max_intensity=150, 
                                    point_radius=5)
figure.add_layer(heatmap_layer)

figure

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
heatmap_df = city_df

heatmap_df = heatmap_df[heatmap_df["Temp in F"]<80]
heatmap_df = heatmap_df[heatmap_df["Temp in F"]>70]
heatmap_df = heatmap_df[heatmap_df["Wind Speed"]<10]
heatmap_df = heatmap_df[heatmap_df["Cloudiness"]<1]

heatmap_df = heatmap_df.dropna(axis='columns',how='any')

heatmap_df.reset_index(inplace=True)

heatmap_df = heatmap_df.drop(columns=['index'])

print(heatmap_df)

    City_ID                  City  Cloudiness Country        Date  Humidity  \
0    104515  Makkah al Mukarramah           0      SA  1604615456        61   
1   1261853         Narasannapeta           0      IN  1604615466        76   
2   3868633              Vallenar           0      CL  1604615340        27   
3   3439297               Caazapá           0      PY  1604615477        30   
4   3464008            Esmeraldas           0      BR  1604615268        47   
5   3386213                Touros           0      BR  1604615486        83   
6   1292037             Thayetmyo           0      MM  1604615487        91   
7    286621               Salalah           0      OM  1604615242        46   
8   1258406                 Rāpar           0      IN  1604615497        64   
9   2065594             Mount Isa           0      AU  1604615501        15   
10   921900              Dzaoudzi           0      YT  1604615536        88   
11  3843123               Neuquén           0      A

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = heatmap_df
hotel_df["Hotel Name"] = ""

In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "rankby": "distance",
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}

hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp in F,Min Temp in F,Wind Speed,Temp in F,Feels Like,Hotel Name
0,104515,Makkah al Mukarramah,0,SA,1604615456,61,21.43,39.83,75.11,75.11,4.47,75.11,76.14,
1,1261853,Narasannapeta,0,IN,1604615466,76,18.42,84.05,74.53,74.53,5.66,74.53,77.29,
2,3868633,Vallenar,0,CL,1604615340,27,-28.57,-70.76,77.45,77.45,6.82,77.45,71.55,
3,3439297,Caazapá,0,PY,1604615477,30,-26.15,-56.40,70.86,70.86,6.71,70.86,64.47,
4,3464008,Esmeraldas,0,BR,1604615268,47,-19.76,-44.31,78.80,77.00,2.24,78.17,78.87,
5,3386213,Touros,0,BR,1604615486,83,-5.20,-35.46,78.80,78.80,9.17,78.80,82.96,
6,1292037,Thayetmyo,0,MM,1604615487,91,19.32,95.18,73.00,73.00,2.35,73.00,79.43,
7,286621,Salalah,0,OM,1604615242,46,17.02,54.09,71.60,71.60,6.93,71.60,67.69,
8,1258406,Rāpar,0,IN,1604615497,64,23.57,70.63,74.52,74.52,2.82,74.52,76.78,
9,2065594,Mount Isa,0,AU,1604615501,15,-20.73,139.50,77.00,77.00,8.05,77.00,68.07,


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [11]:
# Add marker layer ontop of heat map
fig = gmaps.figure(map_type="HYBRID")
marker_layer = gmaps.marker_layer(locations, 
                                  info_box_content=hotel_info)
fig.add_layer(marker_layer)

heat_layer = gmaps.heatmap_layer(locations, 
                                 dissipating=False, 
                                 max_intensity=10,
                                 point_radius=1)
fig.add_layer(heat_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
embed_minimal_html('export.html', views=[fig])